###### Can I predict the existence of subfields with some cool unsupervised learning algorithm? 

For starters, let's just use regular n-grams. A more advanced version would be to look for noun phrases or J&K POS tags.

In [48]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../../')

%matplotlib inline
# import matplotlib.pyplot as plt
import time
import numpy as np
# import scipy as sp
import re
from collections import Counter
import itertools
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.cluster import KMeans
from sklearn.decomposition import SparsePCA
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
# from sklearn import metrics
from sklearn.externals import joblib

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [49]:
# What are the available categories?
categories = sorted([x.name for x in session.query(Category)])
print categories 

[u'acc-phys', u'adap-org', u'alg-geom', u'ao-sci', u'astro-ph', u'astro-ph.CO', u'astro-ph.EP', u'astro-ph.GA', u'astro-ph.HE', u'astro-ph.IM', u'astro-ph.SR', u'atom-ph', u'bayes-an', u'chao-dyn', u'chem-ph', u'comp-gas', u'cond-mat', u'cond-mat.dis-nn', u'cond-mat.mes-hall', u'cond-mat.mtrl-sci', u'cond-mat.other', u'cond-mat.quant-gas', u'cond-mat.soft', u'cond-mat.stat-mech', u'cond-mat.str-el', u'cond-mat.supr-con', u'cs.AI', u'cs.AR', u'cs.CC', u'cs.CE', u'cs.CG', u'cs.CL', u'cs.CR', u'cs.CV', u'cs.CY', u'cs.DB', u'cs.DC', u'cs.DL', u'cs.DM', u'cs.DS', u'cs.ET', u'cs.FL', u'cs.GL', u'cs.GR', u'cs.GT', u'cs.HC', u'cs.IR', u'cs.IT', u'cs.LG', u'cs.LO', u'cs.MA', u'cs.MM', u'cs.MS', u'cs.NA', u'cs.NE', u'cs.NI', u'cs.OH', u'cs.PF', u'cs.PL', u'cs.RO', u'cs.SC', u'cs.SD', u'cs.SE', u'cs.SI', u'cs.SY', u'dg-ga', u'funct-an', u'gr-qc', u'hep-ex', u'hep-lat', u'hep-ph', u'hep-th', u'math-ph', u'math.AC', u'math.AG', u'math.AP', u'math.AT', u'math.CA', u'math.CO', u'math.CT', u'math.CV',

In [50]:
abstract_all_tmp = {'category': [], 'abstract': []}
category_list = sorted(['atom-ph', 'quant-gas']) # THIS WORKS REALLY WELL!
# category_list = sorted(['atom-ph', 'optics',  'quant-gas'])
# category_list = sorted(['atom-ph', 'optics', 'quant-ph', 'quant-gas'])
category_len = len(category_list)

start = time.time()
for item in category_list:
    query = session.query(Article_Category)\
                        .join(Category)\
                        .join(Article)\
                        .filter(Category.name.like('%' + item + '%'))
#     query = session.query(Article_Category)\
#                         .join(Category)\
#                         .join(Article)\
#                         .filter(Category.name.like('%' + item + '%'))
    result = [' '.join(x.article.abstract.split()) for x in query]
    abstract_all_tmp['abstract'].extend(result)
    abstract_all_tmp['category'].extend([item]*len(result))
print time.time() - start
# for item in query:
#     abstract_all['category'].append(item.category.name)
#     abstract_all['abstract'].append(' '.join(item.article.abstract.split()))
# print time.time() - start
# abstract_all['atom-ph'] = [x.article.abstract for x in query.all()]
# session.close_all()

32.3356580734


In [51]:
# Breakdown of categories?
count = Counter(abstract_all_tmp['category'])
for key, val in count.iteritems():
    print '{:<15}{}'.format(key, val)
print '{:<15}{}'.format('Total', len(abstract_all_tmp['abstract']))

quant-gas      9107
atom-ph        9156
Total          18263


In [52]:
# Deduplicate

abstract_all = list(set(abstract_all_tmp['abstract']))
print len(abstract_all)
x_train = abstract_all

17244


In [53]:
x_train = abstract_all
# # Train on 90% of the data. Random_state ensures that we always get the same result.
# x_train, x_test, y_train, y_test = train_test_split(abstract_all['abstract'],
#                                                     abstract_all['category'],
#                                                     random_state=42,
#                                                     train_size=0.9)

# counter_train = Counter(y_train)

Now, try KMeans clustering. 
See: http://scikit-learn.org/stable/auto_examples/text/document_clustering.html

In [58]:
# n_clusters = 20
n_clusters = 25
# 30 is too many!
# Reduce n_init to 10 for testing purposes.
clf_unsupervised = Pipeline([('vect', CountVectorizer(ngram_range=(1,3), stop_words='english')),
                             ('tfidf', TfidfTransformer()),
                             ('clf', KMeans(n_clusters=n_clusters, n_init=10))])
start = time.time()
clf_unsupervised.fit(x_train)
print time.time() - start

start = time.time()
predict_train = clf_unsupervised.predict(x_train)
# predict = clf_unsupervised.predict(x_test)
print time.time() - start
    

804.503778934
6.90848183632


Save the model and predictor

In [59]:
start = time.time()
joblib.dump(clf_unsupervised, 'cluster-AMO.pkl', compress=True) 
# joblib.dump(clf_unsupervised, 'cluster-AMO-optics.pkl', compress=True) 
print time.time() - start

64.7267770767


In [60]:
# Count the number of articles in each chunk
start = time.time()
count_chunks = Counter(clf_unsupervised.predict(x_train))
print time.time() - start

6.95335888863


In [61]:
# Most important chunks. See http://scikit-learn.org/stable/auto_examples/text/document_clustering.html
order_centroids = clf_unsupervised.named_steps['clf'].cluster_centers_.argsort()[:, ::-1]

terms =  clf_unsupervised.named_steps['vect'].get_feature_names()
for i in range(n_clusters):
    print "Cluster %d (%d articles):" % (i, count_chunks[i])
    print ', '.join([terms[x] for x in order_centroids[i, :20]])
    print ''

Cluster 0 (793 articles):
condensate, bose einstein condensate, einstein condensate, einstein, bose einstein, bose, bec, potential, einstein condensate bec, condensate bec, dynamics, equation, phase, quantum, time, field, pitaevskii, trap, gross, gross pitaevskii

Cluster 1 (497 articles):
resonances, molecules, feshbach, resonance, magnetic, ultracold, scattering, state, feshbach resonances, field, bound, dipole, feshbach resonance, atoms, magnetic field, states, molecular, channel, ground, photoassociation

Cluster 2 (355 articles):
spin orbit, orbit, spin, orbit coupling, spin orbit coupling, coupling, soc, rashba, topological, phase, spin orbit coupled, orbit coupled, fermi, superfluid, zeeman, state, coupled, field, zeeman field, momentum

Cluster 3 (392 articles):
scattering, body, scattering length, length, efimov, universal, dimer, range, bound, energy, wave, states, effective, atom, recombination, wave scattering, scattering lengths, state, interactions, trimer

Cluster 4 (349